# Jores et al 2021 Extract-Transform-Load
**Authorship:**
Adam Klie, *08/11/2022*
***
**Description:**
Notebook to extract, transform, and load data from the Jores et al (2021) dataset.
***

In [1]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import os
import pandas as pd
import eugene as eu
eu.settings.dataset_dir = "../../_data/datasets"

Global seed set to 13
2022-08-12 01:06:01.229886: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-12 01:06:01.230032: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Download and load in the dataset to a raw `SeqData` object

In [2]:
# Load in the downloaded datasets from the manuscript Github repo
sdata_leaf_raw = eu.datasets.jores21(dataset="leaf")
sdata_proto_raw = eu.datasets.jores21(dataset="proto")

Dataset jores21 CNN_test_leaf.tsv has already been dowloaded.
Dataset jores21 CNN_train_leaf.tsv has already been dowloaded.
Dataset jores21 CNN_train_proto.tsv has already been dowloaded.
Dataset jores21 CNN_test_proto.tsv has already been dowloaded.


In [3]:
#!wget https://static-content.springer.com/esm/art%3A10.1038%2Fs41477-021-00932-y/MediaObjects/41477_2021_932_MOESM3_ESM.xlsx
!wget https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/analysis/validation_sequences/promoters_for_evolution.tsv
!wget https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/data/misc/CPEs.meme
!wget https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/data/misc/TF-clusters.meme

--2022-08-12 01:11:16--  https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/analysis/validation_sequences/promoters_for_evolution.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57854 (56K) [text/plain]
Saving to: ‘promoters_for_evolution.tsv’

promoters_for_evolu 100%[===================>]  56.50K  --.-KB/s    in 0.004s  

2022-08-12 01:11:16 (13.5 MB/s) - ‘promoters_for_evolution.tsv’ saved [57854/57854]

--2022-08-12 01:11:17--  https://raw.githubusercontent.com/tobjores/Synthetic-Promoter-Designs-Enabled-by-a-Comprehensive-Analysis-of-Plant-Core-Promoters/main/data/misc/CPEs.meme
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.1

In [4]:
# Concatenate the datasets into a combined SeqData object
sdata_combined_raw = eu.dl.concat([sdata_leaf_raw, sdata_proto_raw], keys=["leaf", "proto"])

In [5]:
sdata_combined_raw.seqs_annot.head()

,set,sp,gene,enrichment,batch
seq00000,test,At,AT5G03425,-0.537451,leaf
seq00001,test,Sb,ENSRNA049996419,4.108127,leaf
seq00002,test,Zm,ENSRNA049997541,-0.718419,leaf
seq00003,test,Sb,ENSRNA049996013,1.340581,leaf
seq00004,test,At,AT4G04125,4.227307,leaf


In [6]:
# Load some other sequence features
smetadata = pd.read_excel(
    os.path.join(eu.settings.dataset_dir, "jores21", "41477_2021_932_MOESM3_ESM.xlsx"),
    sheet_name=0, 
    skiprows=3, 
)
smetadata.drop(columns=["species", "gene"], inplace=True)
smetadata["sequence"].isin(sdata_combined_raw.seqs).sum()

76712

In [9]:
# Add the sequence annotations to the combined SeqData object
sdata_combined_raw["sequence"] = sdata_combined_raw.seqs
sdata_combined_raw.seqs_annot = sdata_combined_raw.seqs_annot.merge(smetadata, on="sequence", how="left")
sdata_combined_raw.seqs_annot.drop(columns=["sequence", "UTR"], inplace=True)

In [17]:
# Save raw versions of these three
sdata_leaf_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_raw.h5sd"))
sdata_proto_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_raw.h5sd"))
sdata_combined_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_raw.h5sd"))

# Transform the input data in the combined SeqData object

In [18]:
# Add reverse complement sequences and one-hot encoded sequences (forward and reverse complement)
eu.pp.reverse_complement_data(sdata_combined_raw)
eu.pp.one_hot_encode_data(sdata_combined_raw)

Reverse complementing DNA sequences:   0%|          | 0/147966 [00:00<?, ?it/s]

SeqData object modified:
	rev_seqs: None -> 147966 rev_seqs added


One-hot-encoding sequences:   0%|          | 0/147966 [00:00<?, ?it/s]

One-hot-encoding sequences:   0%|          | 0/147966 [00:00<?, ?it/s]

SeqData object modified:
	ohe_seqs: None -> 147966 ohe_seqs added
	ohe_rev_seqs: None -> 147966 ohe_rev_seqs added


In [19]:
# Save the combined processed SeqData object
sdata_combined_raw.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_processed.h5sd"))

In [20]:
# Subset back to leaf and proto objects
sdata_leaf_processed = sdata_combined_raw[sdata_combined_raw["batch"] == "leaf"]
sdata_proto_processed = sdata_combined_raw[sdata_combined_raw["batch"] == "proto"]

In [21]:
# Split each of the three datases into training and train sets
sdata_leaf_train = sdata_leaf_processed[sdata_leaf_processed["set"] == "train"]
sdata_proto_train = sdata_proto_processed[sdata_proto_processed["set"] == "train"]
sdata_combined_train = sdata_combined_raw[sdata_combined_raw["set"] == "train"]
sdata_leaf_test = sdata_leaf_processed[sdata_leaf_processed["set"] == "test"]
sdata_proto_test = sdata_proto_processed[sdata_proto_processed["set"] == "test"]
sdata_combined_test = sdata_combined_raw[sdata_combined_raw["set"] == "test"]

In [22]:
# Add in a column signifying whether the sequence is in the training set or the validation set
eu.pp.train_test_split_data(sdata_leaf_train, train_key="train_val", split=0.9)
eu.pp.train_test_split_data(sdata_proto_train, train_key="train_val", split=0.9)

SeqData object modified:
    seqs_annot:
        + train_val
SeqData object modified:
    seqs_annot:
        + train_val


In [23]:
# Save all the sets
sdata_leaf_processed.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_processed.h5sd"))
sdata_proto_processed.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_processed.h5sd"))
sdata_leaf_train.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_processed_train.h5sd"))
sdata_proto_train.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_processed_train.h5sd"))
sdata_combined_train.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_processed_train.h5sd"))
sdata_leaf_test.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "leaf_processed_test.h5sd"))
sdata_proto_test.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "proto_processed_test.h5sd"))
sdata_combined_test.write_h5sd(os.path.join(eu.settings.dataset_dir, "jores21", "combined_processed_test.h5sd"))

: 

---

# Scratch